In [19]:
#builds and empty dataframe

import pandas as pd

def buildDF():
    colNames = ["#Eventname", "Date", "Otime", "Lat.", "Long.", "Depth", "Me", "Tr", "Ehf", "Ebb",
                "Mehf", "Ehf/Tr^3", "Nstats : Comments", "TACER_HF", "TACER_BB", "COMMENTS"]
    df = pd.DataFrame(columns = colNames)
    return df

#uses a helper function to input data into the dataframe

def parseRtergHtml(df, html):
    data = readHtml(html)
    for x in range(16):
        df[df.columns[x]] = [data[x]]
    return

#creates a list of data from the html file that will be
#inputted into the dataframe

def readHtml(html):
    file = open(html, "r")
    file.readline()
    line2 = file.readline()
    dataList = line2.split()
    str = ""
    for x in range(3):
        str = str + " " + dataList.pop(13)
    dataList[12] = dataList[12] + str
    line3 = file.readline()
    tempList = line3.split()
    dataList.append(tempList[7])
    dataList.append(tempList[10])
    line4 = file.readline()
    dataList.append(line4[15:].strip())
    return dataList

df = buildDF()
parseRtergHtml(df, "21012303.html")
print(df)

  #Eventname        Date     Otime    Lat.   Long. Depth    Me  Tr      Ehf  \
0   21012303  2021/01/23  23:36:55  -62.00  -55.30  10.0  7.01  53  9.7e+13   

       Ebb  Mehf Ehf/Tr^3      Nstats : Comments TACER_HF TACER_BB  \
0  7.3e+14  6.89  6.5e+08  32 : (SRC=AT, M=7.3);    27.00    26.00   

                   COMMENTS  
0  this is our comment text  


In [4]:
#working on parsing the data and inputting it into the correct rows/columns using pandas.read_csv()

import pandas as pd

def parseRtergHtmlWorking(html):
    colNames1 = ["Eventname", "oDate", "oTime", "Lat.", "Long.", "Depth", "Me", "Txo", "Ehf", "Ebb", "Mehf", "Ehf/Tr^3", "Nstats", "colon", "SRC", "iMag"]
    df1 = pd.read_csv(html, names = colNames1, skiprows = 1, nrows = 1, delim_whitespace=True)
    del df1['colon']  #remove colon column
    
    # keep only first 2 of last 3 characters in SRC
    # look at https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/ for help
    # split iMag  1) everything before '=', 2) after '=' and before ');'
    #         goal is that:
    #                SRC = AT
    #                iMagType = M
    #                iMag = 7.3
    
    colNames2 = ["TACER_HF", "TACER_BB"]
    df2 = pd.read_csv(html, names = colNames2, skiprows = 2, nrows = 1, delim_whitespace=True, usecols=[7,10])
    
    colNames3 = ["junk", "Comment"]
    df3 = pd.read_csv(html, names = colNames3, skiprows = 3, nrows = 1, delimiter=":")
    del df3['junk']
    
    colNames4 = ["mTime"]
    df4 = pd.read_csv(html, names = colNames4, skiprows = 7, nrows = 1, delimiter="?")

    df = pd.concat([df1,df2,df3,df4], axis=1)
    
    # merge oDate oTime to new oTime using the DateTime format
    # convert mTime to DateTime format
    #  This _may_ help: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html 
    
    return df

df =parseRtergHtmlWorking("21012303.html")

In [5]:
df.head()

,Eventname,oDate,oTime,Lat.,Long.,Depth,Me,Txo,Ehf,Ebb,Mehf,Ehf/Tr^3,Nstats,SRC,iMag,TACER_HF,TACER_BB,Comment,mTime
0,21012303,2021/01/23,23:36:55,-62.0,-55.3,10.0,7.01,53,9.700000e+13,7.300000e+14,6.89,650000000.0,32,"(SRC=AT,",M=7.3);,27.0,26.0,this is our comment text,Thu Jan 28 14:25:57 UTC 2021
